In [95]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
from datetime import datetime
import dateutil
import numpy as np



countries = [
    {
        "Type": "Languages",
        "Wiki_URL": "https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory"
        }
]

data = requests.get(countries[0]['Wiki_URL']).text
soup = BeautifulSoup(data,'html.parser')

# find tables on the wiki page
for table in soup.find_all('table'):
    print(table.get('class'))

tables = soup.find_all('table')
table = soup.find('table', class_='wikitable sortable')


df = pd.DataFrame(columns=['Country/Region', 'Official Language', 'Regional Language', 'Minority language', 'National language', 'Widely spoken'])

for row in table.tbody.find_all('tr'):
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        country = columns[0].text.strip()
        off_lang = columns[1].text.strip()
        reg_lang = columns[2].text.strip()
        min_lang = columns[3].text.strip()
        nat_lang = columns[4].text.strip()
        ws_lang = columns[-1].text.strip()

        df = df.append({'Country/Region':country, 'Official Language':off_lang, 'Regional Language': reg_lang, 'Minority language':min_lang, 'National language':nat_lang, 'Widely spoken':ws_lang}, ignore_index=True)

df.head()


['wikitable', 'sortable']
['nowraplinks', 'hlist', 'mw-collapsible', 'autocollapse', 'navbox-inner']
['nowraplinks', 'mw-collapsible', 'expanded', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'mw-collapsible', 'expanded', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']


,Country/Region,Official Language,Regional Language,Minority language,National language,Widely spoken
0,Abkhazia[a],Abkhaz\nRussian,,Georgian,Abkhaz,
1,Afghanistan[1],Persian (Dari)\nPashto,Uzbek[b]\nTurkmen[b]\nPashayi[b]\nNuristani[b]...,,Persian (Dari)\nPashto,Persian (Dari)
2,Albania[2],Albanian,,Greek\nMacedonian\nAromanian,,Italian
3,Algeria[3],Arabic\nTamazight,,,Arabic\nTamazight,French
4,Andorra,Catalan[4],,Spanish\nFrench\nPortuguese,,


In [96]:


old_names = ['Country/Region', 'Official Language', 'Regional Language', 'Minority language', 'National language', 'Widely spoken']
new_names = ['country', 'official_lang', 'regional_lang', 'minority_lang', 'national_lang', 'widely_spoken_lang']

rename_dict = {name[0]:name[1] for name in zip(old_names,new_names)}

df = df.rename(columns=rename_dict)

def country_cleaning(country_name):
    return re.sub(r'([^a-zA-Z])', '', country_name)

df['country'] = df.country.map(country_cleaning)

def lang_clean(langs):
    langs_list_output = []
    langs_list = langs.split("\n")
    for item in langs_list:
        item = re.sub(r'[^a-zA-Z]', '', item)
        langs_list_output.append(item)
    if langs_list_output[0] == '':
        return None
    else:
        return langs_list_output

lang_columns = ['official_lang', 'regional_lang', 'minority_lang', 'national_lang', 'widely_spoken_lang']

for item in lang_columns:
    df[item] = df[item].map(lang_clean)

df.head()

,country,official_lang,regional_lang,minority_lang,national_lang,widely_spoken_lang
0,Abkhaziaa,"[Abkhaz, Russian]",None,[Georgian],[Abkhaz],None
1,Afghanistan,"[PersianDari, Pashto]","[Uzbekb, Turkmenb, Pashayib, Nuristanib, Baloc...",None,"[PersianDari, Pashto]",[PersianDari]
2,Albania,[Albanian],None,"[Greek, Macedonian, Aromanian]",None,[Italian]
3,Algeria,"[Arabic, Tamazight]",None,None,"[Arabic, Tamazight]",[French]
4,Andorra,[Catalan],None,"[Spanish, French, Portuguese]",None,None


In [92]:
df.to_csv('test.csv', header=0)